# ETF Momentum Stratagies
## Paper link
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3468556
## summary
investing in etf, long-term (2~4 years) momentum is efficient and robust.monthly rebalancing

In [1]:
import yfinance as yf
import datetime
import pandas as pd

In [2]:
def generate_link(include_leveraged=False):
    link = f"https://etfdb.com/compare/highest-3-year-returns/"
    if not include_leveraged:
        link = link + "no-leveraged/"
    return link

In [3]:
def sub_year(date, y=0):
    m, d = 0, 0
    date = str(date)[:10]
    ny, nm, nd = [int(x) - y for x, y in zip(date.split("-"), [y, m, d])]
    return pd.to_datetime(f"{ny}-{nm}-{nd}")

In [4]:
def get_delta_price(hist, period=3):
    date = hist.index[-3]
    date = sub_year(date, period)
    old = hist.iloc[list(hist.index).index(date)]
    current = hist.iloc[-1]
    return current["Close"]/old["Close"]

In [5]:
SEED_MONEY = 1178
N = 10
INCLUDE_LEVERAGED = False

In [6]:
link = generate_link(INCLUDE_LEVERAGED)

In [7]:
rank = pd.read_html(link)[0]

In [8]:
top_N = rank.iloc[:N]

In [11]:
total_volume = top_N["Avg Volume"].sum()
weights = []
nodata = []
for i in range(len(top_N.index)):
    df = top_N.iloc[i]
    ticker = yf.Ticker(df["Symbol"])
    hist = ticker.history(period="max")
    try:
        dm = get_delta_price(hist)
    except Exception as e:
        print(e)
        print(f"no data for {df['Symbol']}")
        nodata.append(df['Symbol'])
        continue
    weights.append((dm * df["Avg Volume"])/total_volume)
for l in nodata:
    top_N = top_N.loc[top_N["Symbol"] != l]

Timestamp('2017-05-12 00:00:00') is not in list
no data for ARKG


In [12]:
top_N["weight(%)"] = weights / sum(weights) * 100
top_N["money"] = weights / sum(weights) * SEED_MONEY

In [13]:
top_N

,Symbol,Name,3-Year Return,AUM,Avg Volume,weight(%),money
0,ARKW,ARK Web x.0 ETF,141.21%,"$790,538.18",290517,14.624857,172.280817
2,PALL,Aberdeen Standard Physical Palladium Shares ETF,121.33%,"$236,434.51",59673,2.681043,31.582689
3,ARKK,ARK Innovation ETF,112.94%,"$3,098,650.19",831070,36.576901,430.875888
4,PTH,Invesco DWA Healthcare Momentum ETF,101.28%,"$329,576.55",44070,1.756937,20.696721
5,PSJ,Invesco Dynamic Software ETF,87.96%,"$413,864.32",55959,2.100524,24.744178
6,IGV,iShares Expanded Tech-Software Sector ET,85.64%,"$3,937,984.82",986479,36.668488,431.954784
7,IBUY,Amplify Online Retail ETF,80.24%,"$317,971.38",82149,2.942792,34.666096
8,PTF,Invesco DWA Technology Momentum ETF,80.13%,"$183,044.42",44484,1.608155,18.944065
9,BBC,Virtus LifeSci Biotech Clinical Trials ETF,78.68%,"$37,321.18",29656,1.040302,12.254762
